In [1]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd # this will need to be installed via command line first
import lxml # this will need to be installed via command line first(as well)

In [2]:
# Method to get the salary cap hits for every player currently signed on an NFL team
# Creates a csv with columns player name, salary cap hit, and team location abbrviation 

def GetTeamSalaryCapHits(team):
    # Use requests to get the raw HTML response from overthecap.com
    OTCResponse = requests.get(f'https://overthecap.com/salary-cap/{team[:-4]}')
        
    # If we get a normal response proceed with data scraping
    if OTCResponse.status_code == 200:
        print(f"Request succeeded with status code {OTCResponse.status_code}")
        TeamNameCaptalized = (team[:-4].replace("-", " ")).title()
        print(f"Here are the Salary Cap hits for all players who are currently signed with the {TeamNameCaptalized}:")

        # Pandas read_html method allows the table contents to be put into a DataFrame
        table_tag = 'salary-cap-table contracted-players'
        DataFrames = pd.read_html(OTCResponse.content, attrs = {'class': table_tag})
        
        # Select the specific parts of the dataframe we want, add team location abv to each player
        SalariesDF = DataFrames[0][['Player', 'Cap Number']]
        SalariesDF = SalariesDF.drop(51, axis=0)
        SalariesDF = SalariesDF.assign(TeamLOC = team[-4:].strip("0"))

        # Create a .csv file with all the player names and their cap hits
        SalariesDF.to_csv(f'{TeamNameCaptalized}-Player-Salary-List.csv', index = False, header = False)
        
        # Read the csv back for testing purposes
        SalariesCSV = pd.read_csv(f'{TeamNameCaptalized}-Player-Salary-List.csv', header = None)
        SalariesCSV.columns = ['Player', 'CapNumber', 'TeamLOC']
        print(SalariesCSV)
        

    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {OTCResponse.status_code}")
        return -1


In [3]:
# A list of links to Over The Cap Salary Cap pages for various teams
NFLTeams = ['buffalo-bills BUF', 'miami-dolphins MIA', 'new-england-patriots NE0', 'new-york-jets NYJ',
            'baltimore-ravens BAL', 'cincinnati-bengals CIN', 'cleveland-browns CLE', 'pittsburgh-steelers PIT',
            'houston-texans HOU', 'indianapolis-colts IND', 'jacksonville-jaguars JAX', 'tennessee-titans TEN', 
            'denver-broncos DEN', 'kansas-city-chiefs KC0', 'las-vegas-raiders LV0', 'los-angeles-chargers LAC',
            'dallas-cowboys DAL', 'new-york-giants NYG', 'philadelphia-eagles PHI', 'washington-commanders WSH',
            'chicago-bears CHI', 'detroit-lions DET', 'green-bay-packers GB0', 'minnesota-vikings MIN',
            'atlanta-falcons ATL', 'carolina-panthers CAR', 'new-orleans-saints NO0', 'tampa-bay-buccaneers TB0',
            'arizona-cardinals ARI', 'los-angeles-rams LAR', 'san-francisco-49ers SF0', 'seattle-seahawks SEA'
           ]

# Call the method defined above to get all the player data
def GetMultipleTeamSalaryCapHits(TeamList):
    for team in TeamList:
        GetTeamSalaryCapHits(team)

# GetMultipleTeamSalaryCapHits(NFLTeams)
GetTeamSalaryCapHits('seattle-seahawks SEA')

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Seattle Seahawks:
                 Player    CapNumber TeamLOC
0          D.K. Metcalf  $13,720,000     SEA
1         Quandre Diggs  $11,937,500     SEA
2           Jamal Adams  $11,496,666     SEA
3         Tyler Lockett  $11,060,000     SEA
4            Geno Smith  $10,100,000     SEA
5        Dre'Mont Jones  $10,056,666     SEA
6           Will Dissly   $9,190,000     SEA
7          Bobby Wagner   $7,000,000     SEA
8             Noah Fant   $6,850,000     SEA
9     Devon Witherspoon   $5,792,937     SEA
10        Charles Cross   $4,859,915     SEA
11            Drew Lock   $4,000,000     SEA
12          Julian Love   $3,910,000     SEA
13        Jordyn Brooks   $3,893,147     SEA
14           Devin Bush   $3,500,000     SEA
15      Michael Dickson   $3,300,000     SEA
16         Damien Lewis   $3,274,299     SEA
17          Jason Myers   $3,040,000     SEA
18       